<a href="https://colab.research.google.com/github/Annmodels/mnist_recon/blob/master/CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.ndimage.interpolation

In [0]:
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz

In [0]:
!mkdir MNIST_Fashion
!mv *.gz MNIST_Fashion/

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_Fashion/')

In [0]:
plt.figure(figsize = (1,1))
sample_image = mnist.train.next_batch(1)[0]
sample_image = sample_image.reshape([28,28])
plt.imshow(sample_image , cmap = "gray")
plt.show()

In [0]:
lrate = 0.0002
batch_size = 40
epochs = 124000

In [0]:
image_dim = 784
gen_h1 = 128
disc_h1 = 128

In [0]:
def xavier_init(shape):
  return tf.random_normal(shape = shape , stddev = 1./tf.sqrt(shape[0]/2.0))

In [0]:
def wnb():
  weights = {
      "disc_h1": tf.Variable(xavier_init([image_dim , disc_h1])),
      "disc_final": tf.Variable(xavier_init([disc_h1 , 1])),
      "gen_h1": tf.Variable(xavier_init([image_dim , gen_h1])),
      "gen_final": tf.Variable(xavier_init([gen_h1 , image_dim]))
      }
      
  biases = {
      "disc_h1": tf.Variable(xavier_init([disc_h1])),
      "disc_final": tf.Variable(xavier_init([1])),
      "gen_h1": tf.Variable(xavier_init([gen_h1])),
      "gen_final": tf.Variable(xavier_init([image_dim]))
      }
  return weights,biases

In [0]:
x_a = tf.placeholder(tf.float32 , shape = [None , image_dim])
x_b = tf.placeholder(tf.float32 , shape = [None , image_dim])

In [0]:
weights,bias = wnb()
def DiscriminatorAnB(x):
  hidn_layer = tf.nn.relu(tf.add(tf.matmul(x , weights["disc_h1"]) , bias["disc_h1"]))
  final_layer = tf.add(tf.matmul(hidn_layer , weights["disc_final"]) , bias["disc_final"])
  disc_prob_output = tf.nn.sigmoid(final_layer)
  return disc_prob_output

def GeneratorABnBA(x):
  hidn_layer = tf.nn.relu(tf.add(tf.matmul(x , weights["gen_h1"]) , bias["gen_h1"]))
  final_layer = tf.add(tf.matmul(hidn_layer , weights["gen_final"]) , bias["gen_final"])
  gen_prob_output = tf.nn.sigmoid(final_layer)
  return gen_prob_output

In [0]:
#weights,bias = wnb()
Generator_var = [weights["gen_h1"],weights["gen_final"],bias["gen_h1"],bias["gen_final"]]
Discriminator_var = [weights["disc_h1"],weights["disc_final"],bias["disc_h1"],bias["disc_final"]]

In [0]:
x_ba = GeneratorABnBA(x_b)
disc_a_real = DiscriminatorAnB(x_a)
disc_a_fake = DiscriminatorAnB(x_ba)

x_ab = GeneratorABnBA(x_a)
disc_b_real = DiscriminatorAnB(x_b)
disc_b_fake = DiscriminatorAnB(x_ab)

In [0]:
loss_disc_a = (tf.reduce_mean(tf.square(disc_a_real - tf.ones_like(disc_a_real))) + tf.reduce_mean(tf.square(disc_a_fake)))/2.0
loss_disc_b = (tf.reduce_mean(tf.square(disc_b_real - tf.ones_like(disc_b_real))) + tf.reduce_mean(tf.square(disc_b_fake)))/2.0
Discriminator_Loss = loss_disc_a + loss_disc_b

x_bab = GeneratorABnBA(x_ba)
x_aba = GeneratorABnBA(x_ab)

loss_gen_a = tf.reduce_mean(tf.square(disc_a_fake - tf.ones_like(disc_a_fake)))
loss_gen_b = tf.reduce_mean(tf.square(disc_b_fake - tf.ones_like(disc_b_fake)))
loss_total = loss_gen_a + loss_gen_b

loss_recon_a = tf.reduce_mean(10*tf.abs([x_a - x_aba]))
loss_recon_b = tf.reduce_mean(10*tf.abs([x_b - x_bab]))
loss_recon_total = loss_recon_a + loss_recon_b
Generator_Loss = loss_total + loss_recon_total
               

In [0]:
with tf.name_scope("Optimizer_Discriminator")as scope:
  Discriminator_optimize = tf.train.AdamOptimizer(learning_rate = lrate).minimize(Discriminator_Loss , var_list = Discriminator_var)

with tf.name_scope("Optimizer_Generator")as scope:
  Generator_optimize = tf.train.AdamOptimizer(learning_rate = lrate).minimize(Generator_Loss , var_list = Generator_var)

In [0]:
x_train = mnist.train.images
mid = int(x_train.shape[0]/2)
x_train_real = x_train[:mid]

x_train_rotated = x_train[mid:].reshape(-1,28,28)
x_train_rotated = scipy.ndimage.interpolation.rotate(x_train_rotated,90,axes = (1,2))
x_train_rotated = x_train_rotated.reshape(-1,28*28)


In [0]:
def shuffle_data(x,size):
  start_index = np.random.randint(0,x.shape[0]-size)
  return x[start_index:start_index+size]

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(epochs):
  x_a_batch = shuffle_data(x_train_real,batch_size)
  x_b_batch = shuffle_data(x_train_rotated,batch_size)
  _,Disc_loss_epoch = sess.run([Discriminator_optimize,Discriminator_Loss],feed_dict = {x_a:x_a_batch , x_b:x_b_batch}) 
  _,Gen_loss_epoch = sess.run([Generator_optimize,Generator_Loss],feed_dict = {x_a:x_a_batch , x_b:x_b_batch})
  if epoch%2000 == 0:
    print("steps: {0}  G_loss: {1} , D_loss: {2}".format(epoch,Gen_loss_epoch,Disc_loss_epoch))

In [0]:
n = 6
canvas1 = np.empty((28*n,28*n))
canvas2 = np.empty((28*n,28*n))
for i in range(n):
  test_a = shuffle_data(x_train_real,batch_size)
  test_b = shuffle_data(x_train_rotated,batch_size)
  out_a = sess.run(x_ba , feed_dict = {x_b:test_b})
  out_b = sess.run(x_ab , feed_dict = {x_a:test_a})
  #out_a = -1*(out_a-1)
  #out_b = -1*(out_b-1)
  for j in range(n):
    canvas1[i*28:(i+1)*28 , j*28:(j+1)*28] = out_a[j].reshape([28,28])
  for j in range(n):
    canvas2[i*28:(i+1)*28 , j*28:(j+1)*28] = out_b[j].reshape([28,28])

plt.figure(figsize = (n,n))
plt.imshow(canvas1 , origin = "upper" , cmap = "gray")
plt.show()

plt.figure(figsize = (n,n))
plt.imshow(canvas2 , origin = "upper" , cmap = "gray")
plt.show()